# Example script for Hackathon

Within each cycle of active learning, you can:

1. Collect training data (original training data + your query data).

2. Train a prediction model to predict the DMS_score for each mutant (e.g., M0A).

3. Use the trained model to predict the score for all mutant in the test set.

4. Select query mutants for next round based on certain criteria. You may want to make sure you don't query the same mutant twice as you only have a limited chances of making queries in total.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, Dataset
import random
from copy import deepcopy
import pandas as pd
from scipy.stats import spearmanr
import argparse
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
!pip install -q transformers
!pip install -q accelerate
!pip install -q torch torchvision torchaudio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/MLCB/Hackathon_data

/content/drive/MyDrive/MLCB/Hackathon_data


## 1. collect training data

Upload `sequence.fasta`, `train.csv`, and `test.csv` to the current runtime:

1. click the folder icon on the left

2. click the upload icon and upload the files to the current directory

In [ ]:
with open('sequence.fasta', 'r') as f:
  data = f.readlines()


sequence_wt = data[1].strip()
sequence_wt

'MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLREKMRRRLESGDKWFSLEFFPPRTAEGAVNLISRFDRMAAGGPLYIDVTWHPAGDPGSDKETSSMMIASTAVNYCGLETILHMTCCRQRLEEITGHLHKAKQLGLKNIMALRGDPIGDQWEEEEGGFNYAVDLVKHIRSEFGDYFDICVAGYPKGHPEAGSFEADLKHLKEKVSAGADFIITQLFFEADTFFRFVKACTDMGITCPIVPGIFPIQGYHSLRQLVKLSKLEVPQEIKDVIEPIKDNDAAIRNYGIELAVSLCQELLASGLVPGLHFYTLNREMATTEVLKRLGMWTEDPRRPLPWALSAHPKRREEDVRPIFWASRPKSYIYRTQEWDEFPNGRWGNSSSPAFGELKDYYLFYLKSKSPKEELLKMWGEELTSEESVFEVFVLYLSGEPNRNGHKVTCLPWNDEPLAAETSLLKEELLRVNRQGILTINSQPNINGKPSSDPIVGWGPSGGYVFQKAYLEFFTSRETAEALLQVLKKYELRVNYHLVNVKGENITNAPELQPNAVTWGIFPGREIIQPTVVDPVSFMFWKDEAFALWIERWGKLYEEESPSRTIIQYIHDNYFLVNLVDNDFPLDNCLWQVVEDTLELLNRPTQNARETEAP'

In [ ]:
len(sequence_wt)

656

In [ ]:
656*19


12464

In [ ]:
def get_mutated_sequence(mut, sequence_wt):
  wt, pos, mt = mut[0], int(mut[1:-1]), mut[-1]

  sequence = deepcopy(sequence_wt)

  return sequence[:pos]+mt+sequence[pos+1:]

In [ ]:
query1 = pd.read_csv('query_1.csv')
query1.head(5)

,mutant,DMS_score,sequence
0,K355Q,0.913747,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
1,R356P,0.574405,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
2,R357G,0.378047,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
3,F365E,0.468621,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
4,S372M,0.640720,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...


In [ ]:
query2 = pd.read_csv('query_2.csv')
query2.head(5)

,mutant,DMS_score,sequence
0,G6F,0.796447,MVNEARFNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
1,S9E,0.697405,MVNEARGNSELNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
2,N11M,0.805347,MVNEARGNSSLMPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
3,G16R,0.738921,MVNEARGNSSLNPCLERSASSGSESSKDSSRCSTPGLDPERHERLR...
4,S19P,0.915120,MVNEARGNSSLNPCLEGSAPSGSESSKDSSRCSTPGLDPERHERLR...


In [ ]:
query3 = pd.read_csv('query_3.csv')
query3.head(5)

,mutant,DMS_score,sequence
0,L350E,0.737047,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
1,S351D,0.738005,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
2,H353Q,0.836147,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
3,H353K,0.699721,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
4,R357P,0.681220,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...


In [ ]:
train = pd.read_csv('train.csv')
train

,mutant,DMS_score
0,M0Y,0.2730
1,M0W,0.2857
2,M0V,0.2153
3,M0T,0.3122
4,M0S,0.2180
...,...,...
1135,P347D,0.3876
1136,P347C,0.1837
1137,P347A,0.4611
1138,P347M,0.2412


In [ ]:
# import pandas as pd

# # Read CSV
# strain = pd.read_csv('train.csv')

# # Randomly select 500 rows
# train = strain.sample(n=500, random_state=42)  # Set seed for reproducibility

# # Optional: reset index
# train = train.reset_index(drop=True)

# # Preview
# print(train.head())

In [ ]:
df_test = pd.read_csv('test.csv')


In [ ]:

train['sequence'] = train.mutant.apply(lambda x: get_mutated_sequence(x, sequence_wt))
train

,mutant,DMS_score,sequence
0,M0Y,0.2730,YVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
1,M0W,0.2857,WVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
2,M0V,0.2153,VVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
3,M0T,0.3122,TVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
4,M0S,0.2180,SVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
...,...,...,...
1135,P347D,0.3876,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
1136,P347C,0.1837,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
1137,P347A,0.4611,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
1138,P347M,0.2412,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...


In [ ]:
df_train = pd.concat([train, query1, query2, query3], ignore_index=True)

In [ ]:
df_train

,mutant,DMS_score,sequence
0,M0Y,0.273000,YVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
1,M0W,0.285700,WVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
2,M0V,0.215300,VVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
3,M0T,0.312200,TVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
4,M0S,0.218000,SVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
...,...,...,...
1435,T638R,0.412769,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
1436,E640A,0.966497,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
1437,L641C,0.823889,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
1438,T646F,0.934512,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...


In [ ]:

df_test['sequence'] = df_test.mutant.apply(lambda x: get_mutated_sequence(x, sequence_wt))
df_test

,mutant,sequence
0,V1D,MDNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
1,V1Y,MYNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
2,V1C,MCNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
3,V1A,MANEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
4,V1E,MENEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
...,...,...
11319,P655S,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
11320,P655T,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
11321,P655V,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
11322,P655A,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...


In [ ]:
# TODO: integrate the query data that you acquired each round into df_train

## 2. Train a prediction model

Here, we provided a linear regression model and used one-hot encoding to encode each variant. You would need to build your own model to achieve better performances.

Hint: you can perform cross-validation on the training set to evaluate your predictor before making predictions on the test set.

In [ ]:
df_train.sequence.values


array(['YVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLREKMRRRLESGDKWFSLEFFPPRTAEGAVNLISRFDRMAAGGPLYIDVTWHPAGDPGSDKETSSMMIASTAVNYCGLETILHMTCCRQRLEEITGHLHKAKQLGLKNIMALRGDPIGDQWEEEEGGFNYAVDLVKHIRSEFGDYFDICVAGYPKGHPEAGSFEADLKHLKEKVSAGADFIITQLFFEADTFFRFVKACTDMGITCPIVPGIFPIQGYHSLRQLVKLSKLEVPQEIKDVIEPIKDNDAAIRNYGIELAVSLCQELLASGLVPGLHFYTLNREMATTEVLKRLGMWTEDPRRPLPWALSAHPKRREEDVRPIFWASRPKSYIYRTQEWDEFPNGRWGNSSSPAFGELKDYYLFYLKSKSPKEELLKMWGEELTSEESVFEVFVLYLSGEPNRNGHKVTCLPWNDEPLAAETSLLKEELLRVNRQGILTINSQPNINGKPSSDPIVGWGPSGGYVFQKAYLEFFTSRETAEALLQVLKKYELRVNYHLVNVKGENITNAPELQPNAVTWGIFPGREIIQPTVVDPVSFMFWKDEAFALWIERWGKLYEEESPSRTIIQYIHDNYFLVNLVDNDFPLDNCLWQVVEDTLELLNRPTQNARETEAP',
       'WVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLREKMRRRLESGDKWFSLEFFPPRTAEGAVNLISRFDRMAAGGPLYIDVTWHPAGDPGSDKETSSMMIASTAVNYCGLETILHMTCCRQRLEEITGHLHKAKQLGLKNIMALRGDPIGDQWEEEEGGFNYAVDLVKHIRSEFGDYFDICVAGYPKGHPEAGSFEADLKHLKEKVSAGADFIITQLFFEADTFFRFVKACTDMGITCPIVPGIFPIQGYHSLRQLVKLSKLEVPQEIKDVIEPIKDNDAAIRNYGIELAVSLCQELLASGLVPGLHFYTLNR

In [ ]:
'''hyperparameters'''

seq_length = 656
seed = 0 # seed for splitting the validation set
val_ratio = 0.1 # proportion of validation set

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from transformers import EsmModel, EsmTokenizer
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load tokenizer and model
# model_name = "facebook/esm1v_t33_650M_UR90S_1"v1
# model_name = "facebook/esm1_t6_43M_UR50S" #esm 1
# model_name = "facebook/esm2_t6_8M_UR50D"
model_name = "facebook/esm2_t30_150M_UR50D"

tokenizer = EsmTokenizer.from_pretrained(model_name)
esm_model = EsmModel.from_pretrained(model_name).to(device)
esm_model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/595M [00:00<?, ?B/s]

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t30_150M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


EsmModel(
  (embeddings): EsmEmbeddings(
    (word_embeddings): Embedding(33, 640, padding_idx=1)
    (dropout): Dropout(p=0.0, inplace=False)
    (position_embeddings): Embedding(1026, 640, padding_idx=1)
  )
  (encoder): EsmEncoder(
    (layer): ModuleList(
      (0-29): 30 x EsmLayer(
        (attention): EsmAttention(
          (self): EsmSelfAttention(
            (query): Linear(in_features=640, out_features=640, bias=True)
            (key): Linear(in_features=640, out_features=640, bias=True)
            (value): Linear(in_features=640, out_features=640, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (rotary_embeddings): RotaryEmbedding()
          )
          (output): EsmSelfOutput(
            (dense): Linear(in_features=640, out_features=640, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (LayerNorm): LayerNorm((640,), eps=1e-05, elementwise_affine=True)
        )
        (intermediate): EsmIntermediate(
  

In [ ]:
import numpy as np
from tqdm import tqdm

@torch.no_grad()
def embed_sequence_esm2(sequence: str):
    # Add special tokens and tokenize
    inputs = tokenizer(sequence, return_tensors="pt", add_special_tokens=True).to(device)

    # Get embeddings from ESM
    outputs = esm_model(**inputs)
    token_embeddings = outputs.last_hidden_state.squeeze(0)  # (seq_len + 2, hidden_dim)

    # Remove special tokens (<cls> and <eos>)
    token_embeddings = token_embeddings[1:-1]

    # Convert to numpy
    return token_embeddings.cpu().numpy()  # shape: (L, D)


In [ ]:
import numpy as np
from tqdm import tqdm

@torch.no_grad()
def embed_sequence_esmv1(sequence: str):
    # Add special tokens and tokenize
    inputs = tokenizer(sequence, return_tensors="pt", add_special_tokens=True).to(device)

    # Get embeddings from ESM
    outputs = esm_model(**inputs)
    token_embeddings = outputs.last_hidden_state.squeeze(0)  # (seq_len + 2, hidden_dim)

    # Remove special tokens (<cls> and <eos>)
    token_embeddings = token_embeddings[1:-1]

    # Convert to numpy
    return token_embeddings.cpu().numpy()  # shape: (L, D)


In [ ]:
from torch.nn.utils.rnn import pad_sequence

def pad_collate(batch):
    sequences, targets = zip(*batch)  # Each sequence is (L, D)
    padded = pad_sequence(sequences, batch_first=True)  # (B, L_max, D)
    targets = torch.stack(targets)  # (B,)
    return padded, targets


In [ ]:
import torch
from torch.utils.data import Dataset
from tqdm import tqdm

class ProteinDatasetESM(Dataset):
    def __init__(self, df, istrain=True, normalize=True, use_esm2=True): #model to be use dand normalise
        """
        Args:
            df (pd.DataFrame): Must contain 'sequence' and optionally 'DMS_score'
            istrain (bool): If True, loads targets and computes normalization stats
            normalize (bool): Whether to apply z-score normalization
            use_esm2 (bool): If True, use embed_sequence_esm2, else use embed_sequence_esmv1
        """
        self.df = df.reset_index(drop=True)
        self.istrain = istrain
        self.normalize = normalize
        self.use_esm2 = use_esm2
        self.num_samples = len(df)
        self.normalize = False
        self.embeddings = []
        self.targets = torch.zeros(self.num_samples, dtype=torch.float32)

        print(f"Encoding sequences using {'ESM-2' if use_esm2 else 'ESM-1v'}...")

        # Encode sequences
        if istrain:
            for it, (seq, target) in enumerate(tqdm(self.df[['sequence', 'DMS_score']].values)):
                emb = embed_sequence_esm2(seq) if use_esm2 else embed_sequence_esmv1(seq)
                self.embeddings.append(torch.tensor(emb))  # shape (L, D)
                self.targets[it] = target
        else:
            for it, seq in enumerate(tqdm(self.df['sequence'].values)):
                emb = embed_sequence_esm2(seq) if use_esm2 else embed_sequence_esmv1(seq)
                self.embeddings.append(torch.tensor(emb))
                self.targets[it] = 0.0  # dummy

        # # Compute normalization stats (mean and std over all (L, D) values)
        # if self.normalize:
        #     all_tokens = torch.cat(self.embeddings, dim=0)  # concat over L, shape: (total_L, D)
        #     self.mean = all_tokens.mean(dim=0, keepdim=True)
        #     self.std = all_tokens.std(dim=0, keepdim=True)
        # else:
        #     self.mean = None
        #     self.std = None
        if self.normalize:
            print("here")
            count = 0
            mean = 0
            M2 = 0  # For Welford's algorithm (std computation)

            for emb in self.embeddings:
                batch_size = emb.shape[0]
                count += batch_size

                delta = emb.mean(dim=0, keepdim=True) - mean
                mean += delta * batch_size / count

                # Sum of squared differences for variance (Welford-style)
                M2 += ((emb - mean)**2).sum(dim=0, keepdim=True)

            self.mean = mean
            self.std = torch.sqrt(M2 / count)
        else:
            self.mean = None
            self.std = None


    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        emb = self.embeddings[idx]  # (L, D)

        if self.normalize and self.mean is not None and self.std is not None:
            emb = (emb - self.mean) / (self.std + 1e-6)

        return emb, self.targets[idx]


In [ ]:
11324+1140

12464

DEEP LEARNINF LINEAR LAYER


In [ ]:
from torch.utils.data import Subset, DataLoader
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence

# Define val split ratio and seed
val_ratio = 0.2
seed = 50

# Step 1: Load datasets with full (L, D) embeddings
train_dataset = ProteinDatasetESM(df_train, istrain=True)
test_dataset = ProteinDatasetESM(df_test, istrain=False)

# Step 2: Split by index (cannot use torch.stack on variable-length tensors)
indices = list(range(len(train_dataset)))
train_idx, val_idx = train_test_split(indices, test_size=val_ratio, random_state=seed)

train_subset = Subset(train_dataset, train_idx)
val_subset = Subset(train_dataset, val_idx)


Encoding sequences using ESM-2...


100%|██████████| 1440/1440 [00:55<00:00, 25.88it/s]


Encoding sequences using ESM-2...


100%|██████████| 11324/11324 [07:14<00:00, 26.08it/s]


In [ ]:
# import torch

# def save_protein_dataset(dataset, filename):
#     embeddings = []
#     targets = []

#     for i in range(len(dataset)):
#         emb, target = dataset[i]  # emb: (Lᵢ, D), target: scalar
#         embeddings.append(emb)
#         targets.append(target)

#     targets = torch.tensor(targets, dtype=torch.float32)

#     torch.save({
#         'embeddings': embeddings,   # list of tensors (Lᵢ, D)
#         'targets': targets          # tensor (N,)
#     }, filename)

# # Save training and test datasets
# save_protein_dataset(train_dataset, "protein_train_full.pt")
# save_protein_dataset(test_dataset, "protein_test_full.pt")


In [ ]:
# import numpy as np

# # Get mean-pooled embeddings (L, D) → (D,)
# train_embeddings = np.stack([emb.mean(dim=0).numpy() for emb in train_dataset.embeddings])
# train_targets = train_dataset.targets.numpy()

# test_embeddings = np.stack([emb.mean(dim=0).numpy() for emb in test_dataset.embeddings])
# np.savez("protein_trainq.npz", embeddings=train_embeddings, targets=train_targets)
# np.savez("protein_testq.npz", embeddings=test_embeddings)


In [ ]:
# import numpy as np
# import torch
# from torch.utils.data import TensorDataset, DataLoader, Subset
# from sklearn.model_selection import train_test_split
# from torch.utils.data import Dataset
# val_ratio = 0.2
# seed = 50
# # Load train and test data
# train_data = np.load("protein_trainq.npz")
# test_data = np.load("protein_testq.npz")

# # Extract arrays
# train_embeddings = torch.tensor(train_data['embeddings'], dtype=torch.float32)  # (N, D)
# train_targets = torch.tensor(train_data['targets'], dtype=torch.float32)        # (N,)
# test_embeddings = torch.tensor(test_data['embeddings'], dtype=torch.float32)    # (M, D)


# # Wrap into standard TensorDataset
# full_train_dataset = TensorDataset(train_embeddings, train_targets)

# # Dummy targets for test set if you're only doing prediction
# test_dataset = TensorDataset(test_embeddings, torch.zeros(len(test_embeddings)))
# train_idx, val_idx = train_test_split(
#     np.arange(len(full_train_dataset)), test_size=val_ratio, random_state=seed
# )

# train_subset = Subset(full_train_dataset, train_idx)
# val_subset = Subset(full_train_dataset, val_idx)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

READ EMB

In [ ]:
# from torch.utils.data import Dataset

# class LoadedProteinDataset(Dataset):
#     def __init__(self, embeddings, targets):
#         self.embeddings = embeddings  # list of (Lᵢ, D) tensors
#         self.targets = targets        # tensor of shape (N,)

#     def __len__(self):
#         return len(self.embeddings)

#     def __getitem__(self, idx):
#         return self.embeddings[idx], self.targets[idx]
# # For test set (dummy targets)
# class TestProteinDataset(Dataset):
#     def __init__(self, embeddings):
#         self.embeddings = embeddings

#     def __len__(self):
#         return len(self.embeddings)

#     def __getitem__(self, idx):
#         return self.embeddings[idx], 0.0  # dummy target




In [ ]:
# import numpy as np
# import torch
# from torch.utils.data import TensorDataset, DataLoader, Subset
# from sklearn.model_selection import train_test_split


# import torch
# import numpy as np
# from torch.utils.data import Subset
# from sklearn.model_selection import train_test_split

# # Load the saved train dataset
# train_data = torch.load("protein_train_full.pt")
# embeddings = train_data['embeddings']  # list of (Lᵢ, D) tensors
# targets = train_data['targets']        # (N,) tensor

# # Wrap into a dataset
# train_dataset = LoadedProteinDataset(embeddings, targets)
# # Define split ratio and seed
# val_ratio = 0.2
# seed = 42

# # Generate train/val indices
# train_idx, val_idx = train_test_split(
#     np.arange(len(train_dataset)), test_size=val_ratio, random_state=seed
# )

# # Create subsets
# train_subset = Subset(train_dataset, train_idx)
# val_subset = Subset(train_dataset, val_idx)



# train_idx, val_idx = train_test_split(np.arange(len(train_dataset)), test_size=val_ratio, random_state=seed)
# train_subset = Subset(train_dataset, train_idx)
# val_subset = Subset(train_dataset, val_idx)
# test_data = torch.load("protein_test_full.pt")
# test_dataset = TestProteinDataset(test_data['embeddings'])

In [ ]:
tt, labels = next(iter(train_dataset))

# For a single feature tensor (e.g., image)
print(f"Min: {tt.min()}, Max: {tt.max()}")

Min: -4.195731163024902, Max: 7.552731037139893


In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_subset, batch_size=32, shuffle=True, collate_fn=pad_collate)
val_loader = DataLoader(val_subset, batch_size=32, shuffle=False, collate_fn=pad_collate)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=pad_collate)



In [ ]:
# from torch.nn.utils.rnn import pad_sequence

# def pad_collatet(batch):
#     sequences, targets = zip(*batch)  # Each sequence is (L, D)
#     padded = pad_sequence(sequences, batch_first=True)  # (B, L_max, D)
#     # Wrap individual targets in tensors before stacking
#     targets = torch.stack([torch.tensor([t]) for t in targets])  # (B,)
#     return padded, targets
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=pad_collatet)

In [ ]:
# import torch
# import torch.nn as nn
# from scipy.stats import spearmanr

# # Setup
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# loss_fn = nn.MSELoss()

# # Initialize model
# embedding_dim = next(iter(train_loader))[0].shape[-1]
# model = DMSPredictor(embedding_dim=embedding_dim, hidden_dim=256, num_layers=2, dropout=0.2).to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


DLL

In [ ]:
import torch
import torch.nn as nn

class DMSPredictor(nn.Module):
    def __init__(self, embedding_dim, hidden_dim=256, num_layers=2, dropout=0.2):
        super(DMSPredictor, self).__init__()

        layers = []

        # Input layer
        layers.append(nn.Linear(embedding_dim, hidden_dim))
        layers.append(nn.LayerNorm(hidden_dim))
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(dropout))

        # Hidden layers
        for _ in range(num_layers - 1):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.LayerNorm(hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))

        # Output layer
        layers.append(nn.Linear(hidden_dim, 1))

        self.mlp = nn.Sequential(*layers)

    def forward(self, x):
        """
        x: Tensor of shape (B, L, D) or (B, D)
        """
        if x.ndim == 3:
            x = x.mean(dim=1)  # (B, D)

        return self.mlp(x).squeeze(-1)  # (B,)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DMSPredictorCNN(nn.Module):
    def __init__(self, embedding_dim, conv_layers=[(128, 5), (128, 3)], dropout=0.3, task='regression', num_classes=1):
        """
        Args:
            embedding_dim (int): D, the ESM embedding size per residue.
            conv_layers (list): Each tuple is (num_filters, kernel_size)
            dropout (float): Dropout rate
            task (str): 'regression' or 'classification'
            num_classes (int): Required for classification
        """
        super(DMSPredictorCNN, self).__init__()

        layers = []
        in_channels = embedding_dim

        for out_channels, kernel_size in conv_layers:
            layers.append(nn.Conv1d(in_channels, out_channels, kernel_size, padding=kernel_size // 2))
            layers.append(nn.BatchNorm1d(out_channels))  # 🔥 normalization added
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            in_channels = out_channels

        self.conv_block = nn.Sequential(*layers)
        self.task = task

        # Final linear layer
        if task == 'regression':
            self.output_layer = nn.Linear(in_channels, 1)
        elif task == 'classification':
            self.output_layer = nn.Linear(in_channels, num_classes)
        else:
            raise ValueError("task must be 'regression' or 'classification'")

    def forward(self, x):
        """
        x: Tensor of shape (B, L, D)
        """
        x = x.transpose(1, 2)              # → (B, D, L) for Conv1d
        x = self.conv_block(x)            # → (B, C, L)
        x = F.adaptive_max_pool1d(x, 1)   # → (B, C, 1)
        x = x.squeeze(-1)                 # → (B, C)

        out = self.output_layer(x)        # → (B, 1) or (B, num_classes)
        return out.squeeze(-1) if self.task == 'regression' else out


In [ ]:
conv_layers = [
    (256, 9),   # Layer 1: 256 filters, kernel size 9
    (256, 7),   # Layer 2
    (128, 5),   # Layer 3
    (128, 3),   # Layer 4
    (64, 3),    # Layer 5
    (64, 3)     # Layer 6
]
# conv_layers = [
#     (256, 9),  # Wide first layer for long-range patterns
#     (128, 5),  # Mid-sized kernel
#     (64, 3)    # Narrow layer for fine-grained features
# ]


In [ ]:
# input_dim = next(iter(train_loader))[0].shape[-1]
model = DMSPredictorCNN(
    embedding_dim=640,         # or 320,768 / 1280 depending on your ESM model
    conv_layers=conv_layers,
    dropout=0.1,
    task='regression'          # or 'classification', with num_classes if needed
).to(device)

loss_fn = nn.MSELoss()

LSTM

Train test

In [ ]:
def train_epoch(model, loader, optimizer, loss_fn, device):
    model.train()
    total_loss = 0
    for X, y in loader:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        preds = model(X)
        loss = loss_fn(preds, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * X.size(0)
    return total_loss / len(loader.dataset)

def evaluate(model, loader, device, loss_fn=None):
    model.eval()
    preds_all, targets_all = [], []
    total_loss = 0
    with torch.no_grad():
        for X, y in loader:
            X, y = X.to(device), y.to(device)
            preds = model(X)
            preds_all.extend(preds.cpu().numpy())
            targets_all.extend(y.cpu().numpy())

            if loss_fn:
                loss = loss_fn(preds, y)
                total_loss += loss.item() * X.size(0)

    rho, _ = spearmanr(targets_all, preds_all)
    avg_loss = total_loss / len(loader.dataset) if loss_fn else None
    return rho, avg_loss, preds_all, targets_all


In [ ]:
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, Subset

k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Store fold loaders
fold_loaders = []

for fold, (train_idx, val_idx) in enumerate(kf.split(train_dataset)):
    print(f"Fold {fold + 1}/{k_folds}")

    train_subset = Subset(train_dataset, train_idx)
    val_subset = Subset(train_dataset, val_idx)

    train_loader = DataLoader(train_subset, batch_size=32, shuffle=True, collate_fn=pad_collate)
    val_loader = DataLoader(val_subset, batch_size=32, shuffle=False, collate_fn=pad_collate)

    fold_loaders.append((train_loader, val_loader))


Fold 1/5
Fold 2/5
Fold 3/5
Fold 4/5
Fold 5/5


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

num_epochs = 100
early_stop_patience = 10

for fold, (train_loader, val_loader) in enumerate(fold_loaders):
    print(f"\n🌀 Training on Fold {fold + 1}")

    best_val_loss = float("inf")
    best_val_rho = -1
    best_model_state = None
    patience_counter = 0

    for epoch in range(num_epochs):
        train_loss = train_epoch(model, train_loader, optimizer, loss_fn, device)
        val_rho, val_loss, _, _ = evaluate(model, val_loader, device, loss_fn)

        print(f"Epoch {epoch + 1}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}, Val Spearman = {val_rho:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_val_rho = val_rho
            best_model_state = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter > early_stop_patience:
            print(f"⏹️ Early stopping at epoch {epoch + 1}")
            break



🌀 Training on Fold 1
Epoch 1: Train Loss = 1.5005, Val Loss = 0.6074, Val Spearman = -0.1191
Epoch 2: Train Loss = 0.1648, Val Loss = 0.1406, Val Spearman = -0.2537
Epoch 3: Train Loss = 0.1314, Val Loss = 0.1009, Val Spearman = 0.1634
Epoch 4: Train Loss = 0.1275, Val Loss = 0.1815, Val Spearman = -0.0332
Epoch 5: Train Loss = 0.1227, Val Loss = 0.2516, Val Spearman = -0.2213
Epoch 6: Train Loss = 0.1116, Val Loss = 0.2120, Val Spearman = -0.1408
Epoch 7: Train Loss = 0.1077, Val Loss = 0.2129, Val Spearman = 0.2005
Epoch 8: Train Loss = 0.1049, Val Loss = 0.2768, Val Spearman = -0.0192
Epoch 9: Train Loss = 0.1031, Val Loss = 0.2533, Val Spearman = -0.2298
Epoch 10: Train Loss = 0.1010, Val Loss = 0.2520, Val Spearman = -0.0896
Epoch 11: Train Loss = 0.1000, Val Loss = 0.1919, Val Spearman = 0.1917
Epoch 12: Train Loss = 0.0982, Val Loss = 0.2051, Val Spearman = 0.3109
Epoch 13: Train Loss = 0.1002, Val Loss = 0.2358, Val Spearman = 0.2970
Epoch 14: Train Loss = 0.0961, Val Loss = 0

FINAL RESULTS


In [ ]:
model.eval()
y_test_pred = []

with torch.no_grad():
    for X_batch, _ in test_loader:
        X_batch = X_batch.to(device)
        preds = model(X_batch)
        y_test_pred.extend(preds.cpu().numpy())

In [ ]:
df_test['DMS_score_predicted'] = y_test_pred
df_test



,mutant,sequence,DMS_score_predicted
0,V1D,MDNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...,0.337006
1,V1Y,MYNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...,0.341161
2,V1C,MCNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...,0.333060
3,V1A,MANEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...,0.253757
4,V1E,MENEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...,0.291422
...,...,...,...
11319,P655S,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...,0.378529
11320,P655T,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...,0.292779
11321,P655V,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...,0.380524
11322,P655A,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...,0.470520


In [ ]:
df_test[['mutant', 'DMS_score_predicted']].to_csv('lpredictions.csv')

In [ ]:
df_test

,mutant,sequence,DMS_score_predicted
0,V1D,MDNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...,0.366880
1,V1Y,MYNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...,0.375097
2,V1C,MCNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...,0.378416
3,V1A,MANEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...,0.361663
4,V1E,MENEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...,0.372214
...,...,...,...
11319,P655S,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...,0.363453
11320,P655T,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...,0.357788
11321,P655V,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...,0.362148
11322,P655A,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...,0.369607


In [ ]:
df_test = pd.read_csv("/content/drive/MyDrive/MLCB/Hackathon_data/40_predictions.csv")

## 3. Select query for next round

In [ ]:
df_test.sort_values('DMS_score_predicted', ascending=False).head(100)

,Unnamed: 0,mutant,DMS_score_predicted
11094,11094,N643W,1.164221
8899,8899,L528P,1.108501
8609,8609,E513Y,1.006625
8821,8821,L524H,0.963491
8825,8825,L524Q,0.959799
...,...,...,...
3503,3503,G204P,0.841892
10423,10423,I608D,0.841820
6678,6678,S411L,0.841814
8933,8933,K530V,0.841321


In [ ]:
# # 1. Sort test set by predicted score (descending)
# top10_df = df_test.sort_values('DMS_score_predicted', ascending=False).head(10)

# # 2. Get the 'mutant' column (make sure it's in your df)
# top10_mutants = top10_df['mutant'].tolist()

# # 3. Write to file (one mutant per line)
# with open("top10.txt", "w") as f:
#     for mutant in top10_mutants:
#         f.write(f"{mutant}\n")

In [ ]:
# 1. Sort test set by predicted score (descending)
top10_df = df_test.sort_values('DMS_score_predicted', ascending=False).head(10)
top10_mutants = top10_df['mutant'].tolist()

with open("top10.txt", "w") as f:
    for i, mutant in enumerate(top10_mutants):
        if i < len(top10_mutants) - 1:
            f.write(f"{mutant}\n")
        else:
            f.write(f"{mutant}")  # no newline after last line

In [ ]:
df_test

,mutant,sequence
0,V1D,MDNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
1,V1Y,MYNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
2,V1C,MCNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
3,V1A,MANEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
4,V1E,MENEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
...,...,...
11319,P655S,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
11320,P655T,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
11321,P655V,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
11322,P655A,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...


In [ ]:
# Example: randomly select 100 test variants to be queried.
# Note: random selection may not be a good strategy
# TODO: select query mutants for the next round based on your own criteria

querys = np.random.choice(df_test.mutant.values, size=100, replace=False)
querys


array(['D91E', 'V92H', 'W164L', 'D637S', 'F581D', 'H142D', 'F507H',
       'T652Y', 'W454K', 'Q484H', 'F223N', 'H200N', 'E545R', 'I481N',
       'D281C', 'I76W', 'R294S', 'E184Q', 'S8I', 'V361G', 'G595Y',
       'N549R', 'N455S', 'R51Y', 'W560D', 'D176S', 'G6C', 'C249K',
       'K490C', 'T329D', 'L404E', 'E166Y', 'K145H', 'T652G', 'T646I',
       'A310T', 'S517F', 'L346I', 'M153H', 'A586I', 'A115N', 'L399K',
       'L642M', 'T520P', 'I90A', 'L436P', 'I364K', 'F223H', 'R376D',
       'P453F', 'H612N', 'R265S', 'P159I', 'L533D', 'L322Y', 'Y505V',
       'V473F', 'L270S', 'A557L', 'K272T', 'H538A', 'P442M', 'S602H',
       'A144I', 'D190Y', 'Y188E', 'C630R', 'S429L', 'S218D', 'T243C',
       'Q609N', 'K47W', 'L471S', 'N482R', 'A115Q', 'H95P', 'E599H',
       'K150V', 'G161Y', 'V116L', 'E359L', 'Y610R', 'K400P', 'D288I',
       'E423T', 'D27A', 'A586L', 'Q146R', 'P363G', 'F59K', 'F514A',
       'R49G', 'V497A', 'P551E', 'S493K', 'D176A', 'K371M', 'K467S',
       'P38D', 'L436H'], dtype=obj

K-Fold validation

In [ ]:
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
# loss_fn = nn.MSELoss()

# best_val_rho = -1
# best_model_state = None

# num_epochs = 100
# early_stop_patience = 10
# patience_counter = 0
# for fold, (train_loader, val_loader) in enumerate(fold_loaders):
#     print(f"\n🌀 Training on Fold {fold + 1}")

#     for epoch in range(num_epochs):
#         train_loss = train_epoch(model, train_loader, optimizer, loss_fn, device)
#         val_rho, _, _, _ = evaluate(model, val_loader, device) # unpacks all 4 values
#         print(f"Epoch {epoch + 1}: Train Loss = {train_loss:.4f}, Val Spearman = {val_rho:.4f}")